In [18]:
from openai import api_key
from pydantic import SecretStr
from langchain_openai import ChatOpenAI

api_key = SecretStr(input("Enter your OpenAI API Key:"))

model = ChatOpenAI(model="gpt-4o-mini", api_key=api_key)

# Note: LLMs are not all knowing

They have training cutoff dates, which means they cannot answer questions about events after that date.

The model that we are using (gpt-4o-mini) has a training cutoff of October 2023


In [34]:
model.invoke("What's on the front page of hacker news?").content

"I can't access real-time data or browse the internet, so I can't provide the current front page of Hacker News. However, you can easily check the latest stories by visiting the Hacker News website at news.ycombinator.com. If you're looking for a summary or analysis of common topics discussed on Hacker News, feel free to ask!"

# Enter tool calls

Tool calls allow us to augment the LLM with current information, provide it tools to do meaningful work, or anything else that can be expressed with code

In [35]:
from langchain_core.tools import tool
import requests

@tool
def read_webpage(url: str) -> str:
    """
    Fetch a url's content and return it as a string
    """
    print("Fetching content from", url)
    response = requests.get(url)
    
    return response.text


tools = [read_webpage]

In [36]:
tool_model = model.bind_tools(tools)

In [37]:
from langchain_core.messages import AIMessage
from typing import cast

call: AIMessage = cast(AIMessage, tool_model.invoke("what's on the front page of hacker news?"))

print(repr(call.content))
print(call.tool_calls)

''
[{'name': 'read_webpage', 'args': {'url': 'https://news.ycombinator.com/'}, 'id': 'call_janc1WUF7JlLXheu82d7t3OV', 'type': 'tool_call'}]


# State Machines - Can be thought of as a directed graph

* States (Nodes)
	* The current _state_ of the graph. Dictates what action should be taken
* Transitions (Edges)
	* The list of possible next values for state. Not all transitions are possible.
		* Eg: A traffic light has 3 transitions. Green => Yellow, Yellow => Red and Red => Green. Yellow => Green should be impossible.

In [38]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import HumanMessage, AIMessage
from langgraph.graph import END, START, StateGraph, MessagesState
from langgraph.prebuilt import ToolNode

system_prompt = """
    You must provide your answers in a format readable in a terminal. Do not provide any text other than the answer.
    Your responses must not exceed 120 characters in width but can be multiple lines.
"""


def call_agent(state: MessagesState):
    response = tool_model.invoke([
        ("system", system_prompt),
        *state["messages"]
    ])

    return {
        "messages": [response]
    }


def after_prompt_edge(state: MessagesState):
    if cast(AIMessage, state["messages"][-1]).tool_calls:
        return "tools"
    return END


tool_node = ToolNode(tools)

workflow = StateGraph(MessagesState)

workflow.add_node("prompt", call_agent)

workflow.add_node("tools", tool_node)

workflow.add_edge(START, "prompt")
workflow.add_edge("tools", "prompt")
workflow.add_conditional_edges("prompt", after_prompt_edge)

app = workflow.compile()

# Agentic / Tool Calling Demo

Let's try that first example again, this time with our defined state machine

In [39]:
messages = [
    HumanMessage(content="Return the top 10 stories on hacker news, their titles, URLs, and comment URLs.")
]

final_state = app.invoke(
    {"messages": messages},
)

messages.append(final_state['messages'][-1])

print(final_state['messages'][-1].content)

Fetching content from https://news.ycombinator.com/
```
1. Debugging: Indispensable rules for finding even the most elusive problems (2004)
   URL: https://dwheeler.com/essays/debugging-agans.html
   Comments: https://news.ycombinator.com/item?id=42682602

2. A Laptop Stand Made from a Single Sheet of Recycled Paper
   URL: https://www.core77.com/posts/134948/A-Laptop-Stand-Made-from-a-Single-Sheet-of-Recycled-Paper
   Comments: https://news.ycombinator.com/item?id=42662329

3. I created an open-source Hardware Hacking Wiki – with tutorials for beginners
   URL: https://www.hardbreak.wiki
   Comments: https://news.ycombinator.com/item?id=42672821

4. Subway Stories
   URL: https://subwaystories.nyc/
   Comments: https://news.ycombinator.com/item?id=42684211

5. Lines of code that will beat A/B testing every time (2012)
   URL: https://stevehanov.ca/blog/index.php?id=132
   Comments: https://news.ycombinator.com/item?id=42650954

6. Why Skyscrapers Became Glass Boxes
   URL: https://www

We can even ask follow up questions!

In [40]:

question = input("What is your follow-up question?")
messages.append(HumanMessage(content=question))

final_state = app.invoke(
    {"messages": messages},
)

messages.append(final_state['messages'][-1])

print("You asked:", question)
print(final_state['messages'][-1].content)

Fetching content from https://www.core77.com/posts/134948/A-Laptop-Stand-Made-from-a-Single-Sheet-of-Recycled-Paper
You asked: summarize the laptop stand article
```
The g.stand by grape lab is a laptop stand made from a single sheet of recycled paper. 
Weighing only 45g, it supports heavy laptops and features an origami design for optimal viewing angles and airflow. 
It collapses to 3cm for easy transport and includes a carry case that also functions as a phone stand. 
Priced around $22, it's celebrated for its sustainability and functionality.
```


# Final demo

Because the tool that we gave our agent was general, not specific, we are not limited to fetching results from hacker news!

In [41]:
state = app.invoke({"messages": [
	HumanMessage(content="What are the recent blog posts posted by Source Allies?"),
]})

print(state['messages'][-1].content)

Fetching content from https://sourceallies.com/blog/
1. [Better Nightly Emails through Lambda, SES, and React](https://sourceallies.com/2024/10/better-nightly-emails/)
   - By Paul Rowe | Cloud
   - Building a Lambda in Typescript to pull cost and usage data.

2. [Some fun JavaScript features for API functionality](https://sourceallies.com/2024/10/fun-javascript-features/)
   - By Elias Simpson | Development
   - Making coding more fun with JavaScript.

3. [The Road to Mordor: A Software Engineer’s Guide to Delivering Complex Features](https://sourceallies.com/2024/09/the-road-to-mordor/)
   - By Kyle Hoehns | Development
   - Practical strategies for tackling large projects.

4. [What Is A Delivery Lead?](https://sourceallies.com/2024/08/what-is-a-delivery-lead/)
   - By Whitney Lovelace | Strategy Enablement
   - Exploring the role of a Delivery Lead.

5. [Navigating Agentic AI Reasoning: ReAct vs ReWOO](https://sourceallies.com/2024/08/react-vs-rewoo/)
   - By Source Allies Team | G